# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
student_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
student_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Number of schools
total_schools = len(student_data_complete["school_name"].unique())
#number of students
total_students = len(student_data_complete["Student ID"].unique())
#Sum of budgets
total_budget = sum(student_data_complete["budget"].unique())
#Average math score
avg_math = student_data_complete["math_score"].mean()
#Average reading score
avg_reading = student_data_complete["reading_score"].mean()
#Overall pass rate calculation
overall_pass_rate = (avg_math + avg_reading)/2

#Add a column to the student_data_complete dataframe that says if the student passed
bins = [0, 69, 100]
bin_names = [ "Failed" , "Passed"]
student_data_complete["Passed Math?"] = pd.cut( student_data_complete["math_score"], bins, labels=bin_names )
student_data_complete["Passed Reading?"] = pd.cut( student_data_complete["reading_score"], bins, labels=bin_names )

#Calculate the percent passed for each subject
perc_math = len(student_data_complete.loc[ student_data_complete["Passed Math?"] == "Passed" ]) / total_students
perc_reading = len(student_data_complete.loc[ student_data_complete["Passed Reading?"] == "Passed" ]) / total_students

district_df = pd.DataFrame({"Total Schools":[total_schools], 
                    "Total Students":[total_students],
                    "Total Budget":[total_budget],
                    "Average Math Score":[avg_math],
                    "Average Reading Score":[avg_reading],
                    "% Passing Math":[perc_math],
                    "% Passing Reading":[perc_reading],
                    "% Overall Passing Rate":[overall_pass_rate]})
district_df.head() 

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,0.749809,0.858055,80.431606


In [3]:
student_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Passed Math?,Passed Reading?
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,Passed,Failed
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,Failed,Passed
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,Failed,Passed
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,Failed,Failed
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,Passed,Passed


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
#Create a new data frame starting from just the school data only
school_df = pd.DataFrame(school_data)
#Keep all columns except the School ID in the data frame
school_df = school_df[["school_name","type","size","budget"]]
#Calculate the budget per student in a new column for the data frame
school_df["Per Student Budget"] = school_df["budget"] / school_df["size"]

#Group the student list by school name 
grouped_school = student_data_complete.groupby("school_name")

#Calculate the avg math score per school 
school_math_score = grouped_school["math_score"].mean()
#Define the name attribute of the Series to enable the .join to work
school_math_score = pd.Series(data=school_math_score, name="Average Math Score")

#Calculate the avg reading score per school
school_reading_score = grouped_school["reading_score"].mean()
#Define the name attribute of the Series to enable the .join to work
school_reading_score = pd.Series(data=school_reading_score, name="Average Reading Score")

#DataFrame of only students that passed math
students_passed_math = student_data_complete[(student_data_complete["Passed Math?"] == "Passed")]
#Group this dataframe by school
grouped_passed_math = students_passed_math.groupby("school_name")
#Count the number of students that passed math per school
grouped_count_passed_math = grouped_passed_math["Passed Math?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_math = pd.Series(data=grouped_count_passed_math, name="Students Passed Math")

#DataFrame of only students that passed reading
students_passed_reading = student_data_complete[(student_data_complete["Passed Reading?"] == "Passed")]
#Group this dataframe by school
grouped_passed_reading = students_passed_reading.groupby("school_name")
#Count the number of students that passed reading per school
grouped_count_passed_reading = grouped_passed_reading["Passed Reading?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_reading = pd.Series(data=grouped_count_passed_reading, name="Students Passed Reading")

#Create a new data frame that joins the school and math scores together
school_stats_df = school_df.join(school_math_score, on="school_name", how="outer")
school_stats_df = school_stats_df.join(school_reading_score, on="school_name", how="outer")
school_stats_df = school_stats_df.join(grouped_count_passed_math, on="school_name", how="outer")
school_stats_df = school_stats_df.join(grouped_count_passed_reading, on="school_name", how="outer")

#Calculate Percentage of Students that Passed
perc_math_passed = school_stats_df["Students Passed Math"] / school_stats_df["size"]
school_stats_df["% Students Passed Math"] = (perc_math_passed)*100

#Calculate Percentage of Students that Passed
perc_reading_passed = school_stats_df["Students Passed Reading"] / school_stats_df["size"]
school_stats_df["% Students Passed Reading"] = (perc_reading_passed)*100

#Calculate overall passing rate
school_stats_df["Overall Passing Rate"] = ( school_stats_df["% Students Passed Math"] + school_stats_df["% Students Passed Reading"] ) /2

# #Rename the columns to match the values requested so it is clean
school_stats_df = school_stats_df.rename(columns={"school_name" : "School Name", "type" : "School Type", "size":"Total Students", "budget":"Total School Budget"} )
del school_stats_df["Students Passed Math"]
del school_stats_df["Students Passed Reading"]
school_stats_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Students Passed Math,% Students Passed Reading,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
#Reset index since grouped by
reset_index_df = school_stats_df.reset_index()
reset_index_df.head()

#Set new index to be the School Names
school_index_df = reset_index_df.set_index("School Name")
school_index_df.head()

#Sort the Overall Passing Rate column for the top schools
school_stats_df = school_index_df.sort_values(by=["Overall Passing Rate"],ascending=False)

#Only save and display the top 5 schools
top_5_schools = school_stats_df.head(5)
top_5_schools

,index,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Students Passed Math,% Students Passed Reading,Overall Passing Rate
School Name,,,,,,,,,,
Cabrera High School,6,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,14,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,9,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,4,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,5,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
#Sort the Overall Passing Rate column for the top schools
school_stats_df = school_index_df.sort_values(by=["Overall Passing Rate"],ascending=True)

#Only save and display the top 5 schools
bottom_5_schools = school_stats_df.head(5)
bottom_5_schools

,index,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Students Passed Math,% Students Passed Reading,Overall Passing Rate
School Name,,,,,,,,,,
Rodriguez High School,11,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,1,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,0,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,12,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,13,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
#Create new data frame of only 9th graders
students_9th = student_data_complete[ (student_data_complete["grade"]=="9th")]
#Group the list by School Name
grouped_school = students_9th.groupby("school_name")
#Calculate the average reading score for each school
series_9th = grouped_school["math_score"].mean()
series_9th = pd.Series(data=series_9th, name="9th")

#Create new data frame of only 9th graders
students_10th = student_data_complete[ (student_data_complete["grade"]=="10th")]
#Group the list by School Name
grouped_school = students_10th.groupby("school_name")
#Calculate the average reading score for each school
series_10th = grouped_school["math_score"].mean()
series_10th = pd.Series(data=series_10th, name="10th")

#Create new data frame of only 9th graders
students_11th = student_data_complete[ (student_data_complete["grade"]=="11th")]
#Group the list by School Name
grouped_school = students_11th.groupby("school_name")
#Calculate the average reading score for each school
series_11th = grouped_school["math_score"].mean()
series_11th = pd.Series(data=series_11th, name="11th")

#Create new data frame of only 9th graders
students_12th = student_data_complete[ (student_data_complete["grade"]=="12th")]
#Group the list by School Name
grouped_school = students_12th.groupby("school_name")
#Calculate the average reading score for each school
series_12th = grouped_school["math_score"].mean()
series_12th = pd.Series(data=series_12th, name="12th")

grades = [series_9th, series_10th, series_11th, series_12th]

math_by_grade = pd.concat(grades, axis=1)
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
#Create new data frame of only 9th graders
students_9th = student_data_complete[ (student_data_complete["grade"]=="9th")]
#Group the list by School Name
grouped_school = students_9th.groupby("school_name")
#Calculate the average reading score for each school
series_9th = grouped_school["reading_score"].mean()
series_9th = pd.Series(data=series_9th, name="9th")

#Create new data frame of only 9th graders
students_10th = student_data_complete[ (student_data_complete["grade"]=="10th")]
#Group the list by School Name
grouped_school = students_10th.groupby("school_name")
#Calculate the average reading score for each school
series_10th = grouped_school["reading_score"].mean()
series_10th = pd.Series(data=series_10th, name="10th")

#Create new data frame of only 9th graders
students_11th = student_data_complete[ (student_data_complete["grade"]=="11th")]
#Group the list by School Name
grouped_school = students_11th.groupby("school_name")
#Calculate the average reading score for each school
series_11th = grouped_school["reading_score"].mean()
series_11th = pd.Series(data=series_11th, name="11th")

#Create new data frame of only 9th graders
students_12th = student_data_complete[ (student_data_complete["grade"]=="12th")]
#Group the list by School Name
grouped_school = students_12th.groupby("school_name")
#Calculate the average reading score for each school
series_12th = grouped_school["reading_score"].mean()
series_12th = pd.Series(data=series_12th, name="12th")

grades = [series_9th, series_10th, series_11th, series_12th]

reading_by_grade = pd.concat(grades, axis=1)
reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [49]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#Reset index since grouped by
spend_school_stats = student_data_complete.reset_index()

#Calculate the spend per student and add as a column to our school stats data frame
spend_school_stats["Spend Per Student"] = spend_school_stats["budget"] / spend_school_stats["size"]

spend_school_stats["Spending Ranges (Per Student)"] = pd.cut( spend_school_stats["Spend Per Student"], spending_bins, labels = group_names)

#Group the student list by spend range name 
grouped_spend = spend_school_stats.groupby("Spending Ranges (Per Student)")
grouped_count_students = grouped_spend["size"].count()
# print(grouped_count_students)

#Calculate the avg math score per school 
spend_math_score = grouped_spend["math_score"].mean()
#Define the name attribute of the Series to enable the .join to work
spend_math_score = pd.Series(data=spend_math_score, name="Average Math Score")

#Calculate the avg reading score per school
spend_reading_score = grouped_spend["reading_score"].mean()
#Define the name attribute of the Series to enable the .join to work
spend_reading_score = pd.Series(data=spend_reading_score, name="Average Reading Score")

#Create a new data frame that joins the school and math scores together
spend_stats_df = spend_school_stats.join(spend_math_score, on="Spending Ranges (Per Student)", how="outer")
spend_stats_df = spend_stats_df.join(spend_reading_score, on="Spending Ranges (Per Student)", how="outer")

#DataFrame of only students that passed math
students_passed_math = spend_stats_df[(spend_stats_df["Passed Math?"] == "Passed")]
#Group this dataframe by school
grouped_passed_math = students_passed_math.groupby("Spending Ranges (Per Student)")
#Count the number of students that passed math per school
grouped_count_passed_math = grouped_passed_math["Passed Math?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_math = pd.Series(data=grouped_count_passed_math, name="Students Passed Math")
# print(grouped_count_passed_math)

#DataFrame of only students that passed reading
students_passed_reading = spend_stats_df[(spend_stats_df["Passed Reading?"] == "Passed")]
#Group this dataframe by school
grouped_passed_reading = students_passed_reading.groupby("Spending Ranges (Per Student)")
#Count the number of students that passed reading per school
grouped_count_passed_reading = grouped_passed_reading["Passed Reading?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_reading = pd.Series(data=grouped_count_passed_reading, name="Students Passed Reading")

#Create a new data frame that joins the school and math scores together
spend_stats_df = spend_stats_df.join(spend_math_score, on="Spending Ranges (Per Student)", how="outer", rsuffix="_R")
spend_stats_df = spend_stats_df.join(spend_reading_score, on="Spending Ranges (Per Student)", how="outer",rsuffix="_R")
spend_stats_df = spend_stats_df.join(grouped_count_passed_math, on="Spending Ranges (Per Student)", how="outer")
spend_stats_df = spend_stats_df.join(grouped_count_passed_reading, on="Spending Ranges (Per Student)", how="outer")

grouped_spend2 = spend_stats_df.groupby("Spending Ranges (Per Student)")

#Calculate Percentage of Students that Passed
perc_math_passed = (grouped_count_passed_math / grouped_count_students)*100
spend_stats_df["% Students Passed Math"] = (perc_math_passed)
spend_perc_math_passed = pd.Series(data=perc_math_passed, name="% Students Passed Math")

#Calculate Percentage of Students that Passed
perc_reading_passed = (grouped_count_passed_reading / grouped_count_students)*100
spend_stats_df["% Students Passed Reading"] = (perc_reading_passed)
spend_perc_reading_passed = pd.Series(data=perc_reading_passed, name="% Students Passed Reading")

# #Calculate overall passing rate
overall_passed = ( perc_math_passed + perc_reading_passed ) /2
spend_perc_overall_passed = pd.Series(data=overall_passed, name="% Overall Passed")

# #Rename the columns to match the values requested so it is clean
spend_stats_df = spend_stats_df.rename(columns={"school_name" : "School Name", "type" : "School Type", "size":"Total Students", "budget":"Total School Budget"} )

values = [spend_math_score, spend_reading_score, spend_perc_math_passed, spend_perc_reading_passed, spend_perc_overall_passed]

summary_by_spend = pd.concat(values, axis=1)

summary_by_spend

,Average Math Score,Average Reading Score,% Students Passed Math,% Students Passed Reading,% Overall Passed
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [60]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Reset index since grouped by
size_school_stats = student_data_complete.reset_index()

size_school_stats["School Size"] = pd.cut( size_school_stats["size"], size_bins, labels = group_names)

#Group the student list by size range name 
grouped_size = size_school_stats.groupby("School Size")
grouped_count_students = grouped_size["size"].count()
# print(grouped_count_students)

#Calculate the avg math score per size
size_math_score = grouped_size["math_score"].mean()
#Define the name attribute of the Series to enable the .join to work
size_math_score = pd.Series(data=size_math_score, name="Average Math Score")

#Calculate the avg reading score per size
size_reading_score = grouped_size["reading_score"].mean()
#Define the name attribute of the Series to enable the .join to work
size_reading_score = pd.Series(data=size_reading_score, name="Average Reading Score")

#Create a new data frame that joins the school and math scores together
size_stats_df = size_school_stats.join(size_math_score, on="School Size", how="outer")
size_stats_df = size_stats_df.join(size_reading_score, on="School Size", how="outer")

#DataFrame of only students that passed math
students_passed_math = size_stats_df[(size_stats_df["Passed Math?"] == "Passed")]
#Group this dataframe by school
grouped_passed_math = students_passed_math.groupby("School Size")
#Count the number of students that passed math per school
grouped_count_passed_math = grouped_passed_math["Passed Math?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_math = pd.Series(data=grouped_count_passed_math, name="Students Passed Math")
# print(grouped_count_passed_math)

#DataFrame of only students that passed reading
students_passed_reading = size_stats_df[(size_stats_df["Passed Reading?"] == "Passed")]
#Group this dataframe by school
grouped_passed_reading = students_passed_reading.groupby("School Size")
#Count the number of students that passed reading per school
grouped_count_passed_reading = grouped_passed_reading["Passed Reading?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_reading = pd.Series(data=grouped_count_passed_reading, name="Students Passed Reading")

#Create a new data frame that joins the school and math scores together
size_stats_df = size_stats_df.join(size_math_score, on="School Size", how="outer", rsuffix="_R")
size_stats_df = size_stats_df.join(size_reading_score, on="School Size", how="outer",rsuffix="_R")
size_stats_df = size_stats_df.join(grouped_count_passed_math, on="School Size", how="outer")
size_stats_df = size_stats_df.join(grouped_count_passed_reading, on="School Size", how="outer")

grouped_size2 = size_stats_df.groupby("School Size")

#Calculate Percentage of Students that Passed
perc_math_passed = (grouped_count_passed_math / grouped_count_students)*100
size_stats_df["% Students Passed Math"] = (perc_math_passed)
size_perc_math_passed = pd.Series(data=perc_math_passed, name="% Students Passed Math")

#Calculate Percentage of Students that Passed
perc_reading_passed = (grouped_count_passed_reading / grouped_count_students)*100
size_stats_df["% Students Passed Reading"] = (perc_reading_passed)
size_perc_reading_passed = pd.Series(data=perc_reading_passed, name="% Students Passed Reading")

# #Calculate overall passing rate
overall_passed = ( perc_math_passed + perc_reading_passed ) /2
size_perc_overall_passed = pd.Series(data=overall_passed, name="% Overall Passed")

# #Rename the columns to match the values requested so it is clean
size_stats_df = size_stats_df.rename(columns={"school_name" : "School Name", "type" : "School Type", "size":"Total Students", "budget":"Total School Budget"} )

values = [size_math_score, size_reading_score, size_perc_math_passed, size_perc_reading_passed, size_perc_overall_passed]

summary_by_size = pd.concat(values, axis=1)

summary_by_size

,Average Math Score,Average Reading Score,% Students Passed Math,% Students Passed Reading,% Overall Passed
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [65]:
#Reset index since grouped by
type_school_stats = student_data_complete.reset_index()

#Group the student list by schoool type
grouped_type = type_school_stats.groupby("type")
grouped_count_students = grouped_type["type"].count()
# print(grouped_count_students)

#Calculate the avg math score per school type
type_math_score = grouped_type["math_score"].mean()
#Define the name attribute of the Series to enable the .join to work
type_math_score = pd.Series(data=type_math_score, name="Average Math Score")

#Calculate the avg reading score per school type
type_reading_score = grouped_type["reading_score"].mean()
#Define the name attribute of the Series to enable the .join to work
type_reading_score = pd.Series(data=type_reading_score, name="Average Reading Score")

#Create a new data frame that joins the school and math scores together
type_stats_df = type_school_stats.join(type_math_score, on="type", how="outer")
type_stats_df = type_stats_df.join(type_reading_score, on="type", how="outer")

#DataFrame of only students that passed math
students_passed_math = type_stats_df[(type_stats_df["Passed Math?"] == "Passed")]
#Group this dataframe by school
grouped_passed_math = students_passed_math.groupby("type")
#Count the number of students that passed math per school
grouped_count_passed_math = grouped_passed_math["Passed Math?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_math = pd.Series(data=grouped_count_passed_math, name="Students Passed Math")
# print(grouped_count_passed_math)

#DataFrame of only students that passed reading
students_passed_reading = type_stats_df[(type_stats_df["Passed Reading?"] == "Passed")]
#Group this dataframe by school
grouped_passed_reading = students_passed_reading.groupby("type")
#Count the number of students that passed reading per school
grouped_count_passed_reading = grouped_passed_reading["Passed Reading?"].count()
#Define the name attribute of the Series to enable the .join to work
grouped_count_passed_reading = pd.Series(data=grouped_count_passed_reading, name="Students Passed Reading")

#Create a new data frame that joins the school and math scores together
type_stats_df = type_stats_df.join(type_math_score, on="type", how="outer", rsuffix="_R")
type_stats_df = type_stats_df.join(type_reading_score, on="type", how="outer",rsuffix="_R")
type_stats_df = type_stats_df.join(grouped_count_passed_math, on="type", how="outer")
type_stats_df = type_stats_df.join(grouped_count_passed_reading, on="type", how="outer")

grouped_type2 = type_stats_df.groupby("type")

#Calculate Percentage of Students that Passed
perc_math_passed = (grouped_count_passed_math / grouped_count_students)*100
type_stats_df["% Students Passed Math"] = (perc_math_passed)
type_perc_math_passed = pd.Series(data=perc_math_passed, name="% Students Passed Math")

#Calculate Percentage of Students that Passed
perc_reading_passed = (grouped_count_passed_reading / grouped_count_students)*100
type_stats_df["% Students Passed Reading"] = (perc_reading_passed)
type_perc_reading_passed = pd.Series(data=perc_reading_passed, name="% Students Passed Reading")

# #Calculate overall passing rate
overall_passed = ( perc_math_passed + perc_reading_passed ) /2
type_perc_overall_passed = pd.Series(data=overall_passed, name="% Overall Passed")

# #Rename the columns to match the values requested so it is clean
type_stats_df = type_stats_df.rename(columns={"school_name" : "School Name", "type" : "School Type", "size":"Total Students", "budget":"Total School Budget"} )

values = [type_math_score, type_reading_score, type_perc_math_passed, type_perc_reading_passed, type_perc_overall_passed]

summary_by_type = pd.concat(values, axis=1)

summary_by_type

,Average Math Score,Average Reading Score,% Students Passed Math,% Students Passed Reading,% Overall Passed
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
